In [1]:

import torch
import whisper
import ast
from my_model_daewoung import Mymodel
from data_utils import MinyoDataset, custom_collate_fn, get_wer
from for_test import get_test_data

In [2]:
pre_model = whisper.load_model("/home/daewoong/userdata/danbi/whisper_pretrain/large-v2.pt", device='cpu')  
model_dims = pre_model.dims
model_dims.n_ref_encoder_layer = 3
model_dims.n_ref_decoder_layer = 3
model_dims.n_ref_text_ctx = 1024
model_dims.n_ref_text_state = 1280
model = Mymodel(model_dims)
model.load_state_dict(torch.load('/home/daewoong/userdata/danbi/layer3-3_train_result/val_best_wer_3.pt',map_location = 'cpu'))                           

In [5]:
# temperature를 설정하지 않으면 0, 0.2, 0.4 ... 1.0 까지 돌면서 inference를 진행합니다.
# test set에서 길이를 넘는건 2개 인 것 같습니다.
test_dataset_mid_csv = get_test_data(model, temperature = 0.0)

  0%|          | 0/72 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1819 > 1024). Running this sequence through the model will result in indexing errors


5


Token indices sequence length is longer than the specified maximum sequence length for this model (2128 > 1024). Running this sequence through the model will result in indexing errors


In [7]:
# 저장 할 주소 입력하면 됩니다.
# 현재 오리지널 위스퍼의 결과값은 /home/daewoong/userdata/danbi/csv_list/org_whisper_pred.csv 에 저장되어있습니다.
test_dataset_mid_csv.to_csv('/home/daewoong/userdata/danbi/csv_list/3-3_val_best_wer3.csv')